In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, accuracy_score, f1_score, classification_report, precision_recall_curve
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC, SVR
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

I want to make something like digit classifier type data, where output will be a multiple value

In [2]:
def addingClass(df, className):
    df['Class'] = className
    return df


In [3]:
data = pd.read_csv('data/audi.csv')
data.head()

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,A1,2017,12500,Manual,15735,Petrol,150,55.4,1.4
1,A6,2016,16500,Automatic,36203,Diesel,20,64.2,2.0
2,A1,2016,11000,Manual,29946,Petrol,30,55.4,1.4
3,A4,2017,16800,Automatic,25952,Diesel,145,67.3,2.0
4,A3,2019,17300,Manual,1998,Petrol,145,49.6,1.0


In [4]:
new_data = addingClass(data, 'audi')

In [5]:
new_data['Class'].unique() # Sexy as f...

array(['audi'], dtype=object)

In [6]:
# Now I wanna make a function which can read all the files from data directory...
import os

# path = '/d/Learning/Data Science/Data Analysis/Hands on machine learning with scikit learn and tensorflow/Projects (Self Learning)/Project-8 100k Used Car Dataset/data'
# path = '/data'
path = 'data'
all_files = os.listdir(path=path)

In [7]:
df = pd.DataFrame()

for file in all_files:
    file_name = file[:-4]  # extract the files name so that you can assign the file names as class with each dataframe
    data = pd.read_csv(f'data/{file}')
    data_with_class = addingClass(data, file_name)
    # df.append(data_with_class)
    # print(data_with_class)
    # pd.concat([df, data_with_class])  # it will return a df in return but why you haven't saving it somewhere??
    df = pd.concat([df, data_with_class])


In [8]:
df

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,Class,tax(£),fuel type,engine size,mileage2,fuel type2,engine size2,reference
0,A1,2017.0,12500,Manual,15735,Petrol,150.0,55.4,1.4,audi,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A6,2016.0,16500,Automatic,36203,Diesel,20.0,64.2,2.0,audi,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A1,2016.0,11000,Manual,29946,Petrol,30.0,55.4,1.4,audi,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A4,2017.0,16800,Automatic,25952,Diesel,145.0,67.3,2.0,audi,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A3,2019.0,17300,Manual,1998,Petrol,145.0,49.6,1.0,audi,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15152,Eos,2012.0,5990,Manual,74000,Diesel,125.0,58.9,2.0,vw,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15153,Fox,2008.0,1799,Manual,88102,Petrol,145.0,46.3,1.2,vw,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15154,Fox,2009.0,1590,Manual,70000,Petrol,200.0,42.0,1.4,vw,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15155,Fox,2006.0,1250,Manual,82704,Petrol,150.0,46.3,1.2,vw,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# Ghosh got what I wanted, but looks like some cars have few extra features...

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118150 entries, 0 to 15156
Data columns (total 17 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   model         117995 non-null  object 
 1   year          117903 non-null  float64
 2   price         117995 non-null  object 
 3   transmission  117995 non-null  object 
 4   mileage       117077 non-null  object 
 5   fuelType      108540 non-null  object 
 6   tax           94327 non-null   float64
 7   mpg           99187 non-null   float64
 8   engineSize    108540 non-null  float64
 9   Class         118150 non-null  object 
 10  tax(£)        4860 non-null    float64
 11  fuel type     3517 non-null    object 
 12  engine size   9345 non-null    object 
 13  mileage2      9399 non-null    object 
 14  fuel type2    8537 non-null    object 
 15  engine size2  8537 non-null    object 
 16  reference     9455 non-null    object 
dtypes: float64(5), object(12)
memory usage: 16.2+ MB


In [10]:
# Now that's called real world data full of NaNs and some data... Now this problem has become Multiclass classification problem...

# let's just first place class column in the last

class_col = df[['Class']].values
df = df.drop('Class', axis=1)
# df = pd.concat([df, class_col], axis=1)
type(class_col)

numpy.ndarray

In [11]:
df['Class'] = class_col
df

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,tax(£),fuel type,engine size,mileage2,fuel type2,engine size2,reference,Class
0,A1,2017.0,12500,Manual,15735,Petrol,150.0,55.4,1.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,audi
1,A6,2016.0,16500,Automatic,36203,Diesel,20.0,64.2,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,audi
2,A1,2016.0,11000,Manual,29946,Petrol,30.0,55.4,1.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,audi
3,A4,2017.0,16800,Automatic,25952,Diesel,145.0,67.3,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,audi
4,A3,2019.0,17300,Manual,1998,Petrol,145.0,49.6,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,audi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15152,Eos,2012.0,5990,Manual,74000,Diesel,125.0,58.9,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,vw
15153,Fox,2008.0,1799,Manual,88102,Petrol,145.0,46.3,1.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,vw
15154,Fox,2009.0,1590,Manual,70000,Petrol,200.0,42.0,1.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,vw
15155,Fox,2006.0,1250,Manual,82704,Petrol,150.0,46.3,1.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,vw


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118150 entries, 0 to 15156
Data columns (total 17 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   model         117995 non-null  object 
 1   year          117903 non-null  float64
 2   price         117995 non-null  object 
 3   transmission  117995 non-null  object 
 4   mileage       117077 non-null  object 
 5   fuelType      108540 non-null  object 
 6   tax           94327 non-null   float64
 7   mpg           99187 non-null   float64
 8   engineSize    108540 non-null  float64
 9   tax(£)        4860 non-null    float64
 10  fuel type     3517 non-null    object 
 11  engine size   9345 non-null    object 
 12  mileage2      9399 non-null    object 
 13  fuel type2    8537 non-null    object 
 14  engine size2  8537 non-null    object 
 15  reference     9455 non-null    object 
 16  Class         118150 non-null  object 
dtypes: float64(5), object(12)
memory usage: 16.2+ MB


In [13]:
# Let's check where's NAN is present
# df.iloc[df['model'] == 'NaN', 0]

# df.iloc[:, 0][df.iloc[:, 0].isna()]  # Yeah this is giving me the values of Model where it is NaN
df.iloc[:, :][df.iloc[:, 0].isna()]  # Let's see all the columns


,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,tax(£),fuel type,engine size,mileage2,fuel type2,engine size2,reference,Class
20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unclean cclass
21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unclean cclass
22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unclean cclass
23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unclean cclass
24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unclean cclass
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5087,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unclean focus
5088,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unclean focus
5089,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unclean focus
5090,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unclean focus


In [14]:
df.iloc[:, 0][df.iloc[:, 0].isna()]

20      NaN
21      NaN
22      NaN
23      NaN
24      NaN
       ... 
5087    NaN
5088    NaN
5089    NaN
5090    NaN
5091    NaN
Name: model, Length: 155, dtype: object

Looks like unclean focus class has some issue let's see....

In [15]:
unclean_cclass_df = pd.read_csv('data/unclean cclass.csv')
unclean_focus_df = pd.read_csv('data/unclean focus.csv')

In [16]:
unclean_cclass_df

,model,year,price,transmission,mileage,fuel type,engine size,mileage2,fuel type2,engine size2,reference
0,C Class,2020.0,"£30,495",Automatic,NaN,Diesel,2,"1,200",NaN,NaN,/ad/25017331
1,C Class,2020.0,"£29,989",Automatic,NaN,Petrol,1.5,"1,000",NaN,NaN,/ad/25043746
2,C Class,2020.0,"£37,899",Automatic,NaN,Diesel,2,500,NaN,NaN,/ad/25142894
3,C Class,2019.0,"£30,399",Automatic,NaN,Diesel,2,"5,000",NaN,NaN,/ad/24942816
4,C Class,2019.0,"£29,899",Automatic,NaN,Diesel,2,"4,500",NaN,NaN,/ad/24913660
...,...,...,...,...,...,...,...,...,...,...,...
4001,C Class,2017.0,"£14,700",Manual,"31,357",25,£150,70.6,Diesel,1.598,/ad/25451436
4002,C Class,2018.0,"£18,500",Automatic,"28,248",31,£150,64.2,Diesel,2.143,/ad/25451481
4003,C Class,2014.0,"£11,900",Manual,"48,055",31,£20,65.7,Diesel,2.143,/ad/25057204
4004,C Class,2014.0,"£11,300",Automatic,"49,865",46,£145,56.5,Diesel,2.143,/ad/25144481


In [17]:
unclean_focus_df

,model,year,price,transmission,mileage,fuel type,engine size,mileage2,fuel type2,engine size2,reference
0,Focus,2016.0,"£8,000",Manual,NaN,Petrol,0.999,"38,852",NaN,NaN,/ad/25451236
1,Focus,2019.0,"£13,400",Manual,NaN,Petrol,0.999,"11,952",NaN,NaN,/ad/25466927
2,Focus,2019.0,"£14,600",Manual,NaN,Petrol,1.498,"22,142",NaN,NaN,/ad/25484363
3,Focus,2016.0,"£9,450",Manual,NaN,Diesel,1.56,"14,549",NaN,NaN,/ad/20444868
4,Focus,2015.0,"£9,999",Manual,NaN,Diesel,1.56,"7,010",NaN,NaN,/ad/22551057
...,...,...,...,...,...,...,...,...,...,...,...
5599,Focus,2019.0,"£18,745",Manual,NaN,Diesel,2,"7,855",NaN,NaN,/ad/25149519
5600,Focus,2019.0,"£16,350",Manual,NaN,Petrol,1,"13,891",NaN,NaN,/ad/25149523
5601,Focus,2019.0,"£16,850",Manual,NaN,Petrol,1,"13,452",NaN,NaN,/ad/25149532
5602,Focus,2019.0,"£17,310",Automatic,NaN,Petrol,1,"13,376",NaN,NaN,/ad/25149535


In [18]:
# df.iloc[20:45, :]
unclean_cclass_df[unclean_cclass_df.iloc[:, 0].isna()]

,model,year,price,transmission,mileage,fuel type,engine size,mileage2,fuel type2,engine size2,reference
20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
3392,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3478,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3479,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3524,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
unclean_cclass_df.iloc[:, 0][unclean_cclass_df.iloc[:, 0].isna()].__len__()


99

In [20]:
# Let's see is it really true that all the columns are NaN where model is NaN

# for i, j in enumerate(unclean_cclass_df.columns):
#     print(i, j)  # Best

for i, j in enumerate(unclean_cclass_df.columns):
    print(j, ":", unclean_cclass_df.iloc[:, i]
          [unclean_cclass_df.iloc[:, i].isna()].__len__())


model : 99
year : 102
price : 99
transmission : 99
mileage : 198
fuel type : 2677
engine size : 164
mileage2 : 116
fuel type2 : 198
engine size2 : 198
reference : 99


No It's not, every column has different NaNs count... Well that thing I can also simply find out with .info(), isn't it?

In [21]:
# unclean_cclass_df.info(null_counts=True)  # This isn't working
unclean_cclass_df.isnull().sum()  # That's cool and equivalent to our previous results

model             99
year             102
price             99
transmission      99
mileage          198
fuel type       2677
engine size      164
mileage2         116
fuel type2       198
engine size2     198
reference         99
dtype: int64

`Let's get back to our previous dataset`

In [22]:
#let's look at the big picture now...

df.isnull().sum()

model              155
year               247
price              155
transmission       155
mileage           1073
fuelType          9610
tax              23823
mpg              18963
engineSize        9610
tax(£)          113290
fuel type       114633
engine size     108805
mileage2        108751
fuel type2      109613
engine size2    109613
reference       108695
Class                0
dtype: int64

In [23]:
df['reference'].value_counts()

/ad/25017331    1
/ad/25244988    1
/ad/25323124    1
/ad/25224711    1
/ad/25473007    1
               ..
/ad/25483801    1
/ad/25483807    1
/ad/25127708    1
/ad/25127718    1
/ad/25149536    1
Name: reference, Length: 9455, dtype: int64

#### let's check whether the NaNs values has specific class or not. I mean is it the case that all of the NaNs present in a class or two.

In [24]:
# df.iloc[:,:][df.iloc[:, 0].isna()]
df.iloc[:,:][df.iloc[:, 0].isna()].sum()

model                                                           0
year                                                          0.0
price                                                           0
transmission                                                    0
mileage                                                         0
fuelType                                                        0
tax                                                           0.0
mpg                                                           0.0
engineSize                                                    0.0
tax(£)                                                        0.0
fuel type                                                       0
engine size                                                     0
mileage2                                                        0
fuel type2                                                      0
engine size2                                                    0
reference 

In [25]:
df.iloc[:,:][df.iloc[:, 1].isna()].sum()

C:\Users\knowl\AppData\Local\Temp\ipykernel_22336\1423055735.py:1: FutureWarning: The default value of numeric_only in DataFrame.sum is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.iloc[:,:][df.iloc[:, 1].isna()].sum()


year                                                        0.0
fuelType                                                      0
tax                                                         0.0
mpg                                                         0.0
engineSize                                                  0.0
tax(£)                                                      0.0
Class         unclean cclassunclean cclassunclean cclassuncl...
dtype: object

In [26]:
df.iloc[:,:][df.iloc[:, 2].isna()].sum()


model                                                           0
year                                                          0.0
price                                                           0
transmission                                                    0
mileage                                                         0
fuelType                                                        0
tax                                                           0.0
mpg                                                           0.0
engineSize                                                    0.0
tax(£)                                                        0.0
fuel type                                                       0
engine size                                                     0
mileage2                                                        0
fuel type2                                                      0
engine size2                                                    0
reference 

In [27]:
df.iloc[:,:][df.iloc[:, 3].isna()].sum()

model                                                           0
year                                                          0.0
price                                                           0
transmission                                                    0
mileage                                                         0
fuelType                                                        0
tax                                                           0.0
mpg                                                           0.0
engineSize                                                    0.0
tax(£)                                                        0.0
fuel type                                                       0
engine size                                                     0
mileage2                                                        0
fuel type2                                                      0
engine size2                                                    0
reference 

In [28]:
df.iloc[:,:][df.iloc[:, 4].isna()].sum()

C:\Users\knowl\AppData\Local\Temp\ipykernel_22336\3068609727.py:1: FutureWarning: The default value of numeric_only in DataFrame.sum is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.iloc[:,:][df.iloc[:, 4].isna()].sum()


year                                                    1844317.0
mileage                                                         0
fuelType                                                        0
tax                                                           0.0
mpg                                                           0.0
engineSize                                                    0.0
tax(£)                                                        0.0
fuel type2                                                      0
engine size2                                                    0
Class           unclean cclassunclean cclassunclean cclassuncl...
dtype: object

In [29]:
# Let's see if the Classes are still same if we check those columns where NaNs counts are very high... Column 9 to 15

df.iloc[:,:][df.iloc[:, 9].isna()]

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,tax(£),fuel type,engine size,mileage2,fuel type2,engine size2,reference,Class
0,A1,2017.0,12500,Manual,15735,Petrol,150.0,55.4,1.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,audi
1,A6,2016.0,16500,Automatic,36203,Diesel,20.0,64.2,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,audi
2,A1,2016.0,11000,Manual,29946,Petrol,30.0,55.4,1.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,audi
3,A4,2017.0,16800,Automatic,25952,Diesel,145.0,67.3,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,audi
4,A3,2019.0,17300,Manual,1998,Petrol,145.0,49.6,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,audi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15152,Eos,2012.0,5990,Manual,74000,Diesel,125.0,58.9,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,vw
15153,Fox,2008.0,1799,Manual,88102,Petrol,145.0,46.3,1.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,vw
15154,Fox,2009.0,1590,Manual,70000,Petrol,200.0,42.0,1.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,vw
15155,Fox,2006.0,1250,Manual,82704,Petrol,150.0,46.3,1.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,vw


In [30]:
# let's see the class which has tax (pound sign), fuel type, engine size, mileage2, fuel type2, engine size2, reference value present...

df[df['tax(£)'].notnull()] # Nice nice, getting a dataset where tax (pound) is present, let's check the classes


,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,tax(£),fuel type,engine size,mileage2,fuel type2,engine size2,reference,Class
0,I20,2017.0,7999,Manual,17307,Petrol,NaN,58.9,1.2,145.0,NaN,NaN,NaN,NaN,NaN,NaN,hyundi
1,Tucson,2016.0,14499,Automatic,25233,Diesel,NaN,43.5,2.0,235.0,NaN,NaN,NaN,NaN,NaN,NaN,hyundi
2,Tucson,2016.0,11399,Manual,37877,Diesel,NaN,61.7,1.7,30.0,NaN,NaN,NaN,NaN,NaN,NaN,hyundi
3,I10,2016.0,6499,Manual,23789,Petrol,NaN,60.1,1.0,20.0,NaN,NaN,NaN,NaN,NaN,NaN,hyundi
4,IX35,2015.0,10199,Manual,33177,Diesel,NaN,51.4,2.0,160.0,NaN,NaN,NaN,NaN,NaN,NaN,hyundi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4855,I30,2016.0,8680,Manual,25906,Diesel,NaN,78.4,1.6,0.0,NaN,NaN,NaN,NaN,NaN,NaN,hyundi
4856,I40,2015.0,7830,Manual,59508,Diesel,NaN,65.7,1.7,30.0,NaN,NaN,NaN,NaN,NaN,NaN,hyundi
4857,I10,2017.0,6830,Manual,13810,Petrol,NaN,60.1,1.0,20.0,NaN,NaN,NaN,NaN,NaN,NaN,hyundi
4858,Tucson,2018.0,13994,Manual,23313,Petrol,NaN,44.8,1.6,145.0,NaN,NaN,NaN,NaN,NaN,NaN,hyundi


In [31]:
df[df['tax(£)'].notnull()]['Class'].value_counts()

hyundi    4860
Name: Class, dtype: int64

In [32]:
# Noting it down inside OneNote page, so that I can see on my right side, no need to scroll up and down

# Let's check all other columns too...
df[df['fuel type'].notnull()]['Class'].value_counts()

unclean focus     2188
unclean cclass    1329
Name: Class, dtype: int64

In [33]:
df[df['engine size'].notnull()]['Class'].value_counts()

unclean focus     5503
unclean cclass    3842
Name: Class, dtype: int64

In [34]:
df[df['mileage2'].notnull()]['Class'].value_counts()

unclean focus     5509
unclean cclass    3890
Name: Class, dtype: int64

In [35]:
df[df['fuel type2'].notnull()]['Class'].value_counts()

unclean focus     4729
unclean cclass    3808
Name: Class, dtype: int64

In [36]:
df[df['engine size2'].notnull()]['Class'].value_counts()

unclean focus     4729
unclean cclass    3808
Name: Class, dtype: int64

In [37]:
df[df['reference'].notnull()]['Class'].value_counts()

unclean focus     5548
unclean cclass    3907
Name: Class, dtype: int64

In [38]:
# Let's also see the uncommon ones (features which has pretty less NaNs)

df[df['model'].isnull()]['Class'].value_counts()  # instead of not null, check where it is null...

unclean cclass    99
unclean focus     56
Name: Class, dtype: int64

In [39]:
df[df['year'].isnull()]['Class'].value_counts()

unclean focus     145
unclean cclass    102
Name: Class, dtype: int64

In [40]:
df[df['price'].isnull()]['Class'].value_counts()

unclean cclass    99
unclean focus     56
Name: Class, dtype: int64

In [41]:
df[df['transmission'].isnull()]['Class'].value_counts()

unclean cclass    99
unclean focus     56
Name: Class, dtype: int64

In [42]:
df[df['mileage'].isnull()]['Class'].value_counts()

unclean focus     875
unclean cclass    198
Name: Class, dtype: int64

In [43]:
df[df['fuelType'].isnull()]['Class'].value_counts()

unclean focus     5604
unclean cclass    4006
Name: Class, dtype: int64

In [44]:
df[df['tax'].isnull()]['Class'].value_counts()

unclean focus     5604
focus             5454
hyundi            4860
unclean cclass    4006
cclass            3899
Name: Class, dtype: int64

In [45]:
df[df['mpg'].isnull()]['Class'].value_counts()

unclean focus     5604
focus             5454
unclean cclass    4006
cclass            3899
Name: Class, dtype: int64

In [46]:
df[df['engineSize'].isnull()]['Class'].value_counts()

unclean focus     5604
unclean cclass    4006
Name: Class, dtype: int64

#### Summary of notnulls and nulls...

			a. df[df['tax(£)'].notnull()]['Class'].value_counts()
				hyundi 4860
				
			b. df[df['fuel type'].notnull()]['Class'].value_counts()
				unclean focus: 2188, unclean cclass: 1329
				
			c. df[df['engine size'].notnull()]['Class'].value_counts()
				unclean focus: 5503, unclear cclass: 3842
				
			d. df[df['mileage2'].notnull()]['Class'].value_counts()
				unclean focus: 5509, unclean cclass: 3890
				
			e. df[df['fuel type2'].notnull()]['Class'].value_counts()
				unclean focus: 4729, unclean cclass: 3808
				
			f. df[df['engine size2'].notnull()]['Class'].value_counts()
				unclean focus: 4729, unclean cclass: 3808
				
			g. df[df['reference'].notnull()]['Class'].value_counts()
				unclean focus: 5548, unclean cclass: 3907
			
			h. df[df['model'].isnull()]['Class'].value_counts()  # instead of not null, check where it is null...
				unclean cclass: 99, unclean focus: 56
				
			i. df[df['year'].isnull()]['Class'].value_counts()
				unclean focus: 145, unclean cclass: 102
				
			j. df[df['price'].isnull()]['Class'].value_counts()
				unclean cclass: 99, unclean focus: 56
				
			k. df[df['transmission'].isnull()]['Class'].value_counts()
				unclean cclass: 99, unclean focus: 56
				
			l. df[df['mileage'].isnull()]['Class'].value_counts()
				unclean focus: 875, unclean cclass: 198
				
			m. df[df['fuelType'].isnull()]['Class'].value_counts()
				unclean focus: 5604, unclean cclass: 4006
				
			n. df[df['tax'].isnull()]['Class'].value_counts()
				unclean focus: 5604, focus: 5454, hyundi: 4860, unclean cclass: 4006, cclass: 3899
				
			o. df[df['mpg'].isnull()]['Class'].value_counts()
				unclean focus: 5604, focus: 5454, unclean cclass: 4006, cclass: 3899
				
			p. df[df['engineSize'].isnull()]['Class'].value_counts()
        unclean focus: 5604, unclean cclass: 4006

In [47]:
# Looks like unclean focus and unclean cclass is quite disturbed...
df['Class'].value_counts()

ford              17965
vw                15157
vauxhall          13632
merc              13119
bmw               10781
audi              10668
toyota             6738
skoda              6267
unclean focus      5604
focus              5454
hyundi             4860
unclean cclass     4006
cclass             3899
Name: Class, dtype: int64

In [48]:
correlation = df.corr()
# correlation['Class'].sort_values(ascending=False)
correlation

# I've to convert the data into trainable data then I may able to understand the correlation and all things... But what about visualization using graphs...

C:\Users\knowl\AppData\Local\Temp\ipykernel_22336\493661717.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation = df.corr()


,year,tax,mpg,engineSize,tax(£)
year,1.000000,0.192472,-0.132791,-0.044891,0.271535
tax,0.192472,1.000000,-0.451656,0.281768,NaN
mpg,-0.132791,-0.451656,1.000000,-0.248210,-0.451991
engineSize,-0.044891,0.281768,-0.248210,1.000000,0.224927
tax(£),0.271535,NaN,-0.451991,0.224927,1.000000


#### Preprocessing the data before visualization

Preprocessing refers to the steps taken to clean, transform, and prepare raw data for analysis. This can include handling missing or null values, transforming data into a suitable format, scaling or normalizing features, encoding categorical variables, and other similar tasks. The purpose of preprocessing is to make the data suitable for analysis and to ensure that the results are accurate and reliable.

In [49]:
# Let's preprocess the data😀😀😀

df

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,tax(£),fuel type,engine size,mileage2,fuel type2,engine size2,reference,Class
0,A1,2017.0,12500,Manual,15735,Petrol,150.0,55.4,1.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,audi
1,A6,2016.0,16500,Automatic,36203,Diesel,20.0,64.2,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,audi
2,A1,2016.0,11000,Manual,29946,Petrol,30.0,55.4,1.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,audi
3,A4,2017.0,16800,Automatic,25952,Diesel,145.0,67.3,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,audi
4,A3,2019.0,17300,Manual,1998,Petrol,145.0,49.6,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,audi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15152,Eos,2012.0,5990,Manual,74000,Diesel,125.0,58.9,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,vw
15153,Fox,2008.0,1799,Manual,88102,Petrol,145.0,46.3,1.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,vw
15154,Fox,2009.0,1590,Manual,70000,Petrol,200.0,42.0,1.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,vw
15155,Fox,2006.0,1250,Manual,82704,Petrol,150.0,46.3,1.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,vw


In [50]:
df[df['engine size2'].notnull()]['engine size2']  # These are float dtype, why it is showing they are object dtype???

10       1.6
11       2.1
12       2.1
13       2.1
14       2.1
        ... 
5562    1000
5563    1560
5564    1000
5565    1.56
5585       1
Name: engine size2, Length: 8537, dtype: object

In [51]:
df[df['fuelType'].notnull()]['fuelType'].unique(), df[df['engine size2'].notnull()]['engine size2'].unique(), df[df['fuel type2'].notnull()]['fuel type2'].unique()

(array(['Petrol', 'Diesel', 'Hybrid', 'Other', 'Electric'], dtype=object),
 array(['1.6', '2.1', '2143', '2', '4', '1991', '1600', '1595', '2000',
        '6.2', '1.5', '2145', '1950', '1796', '3982', '3', '1598', '2996',
        '2100', '1597', '1.8', '1497', '1.991', '0', '2148', '2.14',
        '2.143', '3498', '3000', '2.7', '2685', '2.2', 'Unknown', '2.996',
        '2962', '1998', '1461', '3606', '4000', '1.598', '2497', '2987',
        '1.497', '1500', '2.15', '5439', '2597', '1800', '6208', '0.999',
        '1.499', '1.596', '1.997', '1', '1000', '1.498', '1.56', '1999',
        '2.3', '1596', '1798', '999', '1560', '1498', '998', '1997',
        '1499', '1560 cc', '1988', '2521', '2261', '2300', '0.998', '1496',
        '1753', '1989', '2.5', '1596 cc', '2.29', '2.521', '1388', '1.0T'],
       dtype=object),
 array(['Petrol', 'Diesel', 'Hybrid', 'Other'], dtype=object))

In [52]:
for i in df['engine size2'].value_counts():
    print(i)

1972
1929
1636
985
459
174
155
155
109
100
78
67
63
58
56
43
40
38
30
28
27
26
25
23
18
18
17
15
15
13
13
13
10
9
8
7
7
6
6
5
5
5
4
4
4
4
3
3
3
3
3
3
3
2
2
2
2
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [53]:
# Thinking Part is always happening inside the notes...
df['tax(£)'].value_counts()

145.0    2259
30.0      674
150.0     586
20.0      307
135.0     247
125.0     233
0.0       136
200.0      99
160.0      98
235.0      35
165.0      32
140.0      32
205.0      31
260.0      23
240.0      19
265.0      15
300.0      14
325.0       9
330.0       4
305.0       2
120.0       2
190.0       1
555.0       1
115.0       1
Name: tax(£), dtype: int64

In [54]:
# let's check the unique values in bigger columns:

df.columns

Index(['model', 'year', 'price', 'transmission', 'mileage', 'fuelType', 'tax',
       'mpg', 'engineSize', 'tax(£)', 'fuel type', 'engine size', 'mileage2',
       'fuel type2', 'engine size2', 'reference', 'Class'],
      dtype='object')

In [55]:
arr = ['model', 'year', 'price', 'transmission', 'mileage', 'fuelType', 'tax',
       'mpg', 'engineSize']

for i in arr:
    print(f"{i}'s unqiue value: {df[i].unique().__len__()}")

model's unqiue value: 196
year's unqiue value: 30
price's unqiue value: 16440
transmission's unqiue value: 5
mileage's unqiue value: 49211
fuelType's unqiue value: 6
tax's unqiue value: 49
mpg's unqiue value: 209
engineSize's unqiue value: 43


In [56]:
df[df['Class'] == 'unclean cclass']['model'].isna().sum(),df[df['Class'] == 'unclean focus']['model'].isna().sum()

(99, 56)

In [57]:
df[df['Class'] == 'unclean focus']['model'].value_counts(), df[df['Class'] == 'unclean cclass']['model'].value_counts()

( Focus    5548
 Name: model, dtype: int64,
  C Class    3907
 Name: model, dtype: int64)

In [58]:
# df[df['Class'] == 'unclean cclass']['model'].fillna('Focus', inplace=True)
# df[df['Class'] == 'unclean focus']['model'].fillna('C Class', inplace=True)
# df[df['Class'] == 'unclean focus'] = df[df['Class'] == 'unclean focus']['model'].fillna('C Class')
# df[df['Class'] == 'unclean focus']['model']

df['model'] = np.where(df['Class'] == 'unclean cclass', 'Focus', df['model'])
df['model'] = np.where(df['Class'] == 'unclean focus', 'C Class', df['model'])

# df['model'] = np.where(df['Class'] == 'unclean cclass', 'Focus', np.where(df['Class'] == 'unclean focus', 'C Class', df['model']))   # It's the shortcut of above two but it's not readable so yeah didn't use it...

Well that's kind of thing I wanted to do, but didn't know how to do it:

  def get_value_for_a(row):
    if row['B'] == 'x':
        return 0
    elif row['B'] == 'y':
        return 1
    else:
        return row['A']

df['A'] = df.apply(get_value_for_a, axis=1)


In [59]:
df['model'].isna().sum()

0

#### Year Column

In [68]:
# Year Column
df[df['Class'] == 'unclean cclass']['year'].value_counts()

2019.0    1590
2017.0     666
2016.0     545
2018.0     362
2015.0     321
2014.0     136
2020.0     130
2013.0      64
2012.0      26
2011.0      13
2010.0      10
2008.0       8
2007.0       7
2009.0       7
2005.0       5
2002.0       4
2006.0       3
2004.0       3
1991.0       1
1995.0       1
2003.0       1
1998.0       1
Name: year, dtype: int64

In [71]:
df[df['Class'] == 'unclean focus']['year'].value_counts()

2019.0    1594
2017.0    1288
2018.0    1066
2016.0     553
2015.0     393
2014.0     156
2013.0     152
2020.0      66
2012.0      42
2009.0      37
2011.0      32
2008.0      26
2010.0      20
2007.0      15
2005.0      11
2003.0       4
2006.0       3
2002.0       1
Name: year, dtype: int64

Now here is the magic, the unqiue value counts is high, how you gonna fill those nas...

In [77]:
sum_of_cols = df[df['Class'] == 'unclean focus']['year'].value_counts().sum()
count_of_nas = df[df['Class'] == 'unclean focus']['year'].isna().sum()
year_unclean_focus = df[df['Class'] == 'unclean focus']['year'].value_counts()

In [82]:
year_unclean_focus * count_of_nas/sum_of_cols

2019.0    42.339256
2017.0    34.211394
2018.0    28.314710
2016.0    14.688588
2015.0    10.438725
2014.0     4.143616
2013.0     4.037369
2020.0     1.753068
2012.0     1.115589
2009.0     0.982781
2011.0     0.849973
2008.0     0.690603
2010.0     0.531233
2007.0     0.398425
2005.0     0.292178
2003.0     0.106247
2006.0     0.079685
2002.0     0.026562
Name: year, dtype: float64

In [92]:
np.floor(year_unclean_focus * count_of_nas/sum_of_cols).sum(), np.ceil(year_unclean_focus * count_of_nas/sum_of_cols).sum(), count_of_nas

(138.0, 156.0, 145)

Now how am I gonna handle 7 (145-138) values now.... Will fill the rest values with highest occured value in the column which is 2019...

<!-- Well let's also check if there is other value who is correlated to year -->

In [96]:
df[df['year'].isna()]

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,tax(£),fuel type,engine size,mileage2,fuel type2,engine size2,reference,Class
20,Focus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unclean cclass
21,Focus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unclean cclass
22,Focus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unclean cclass
23,Focus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unclean cclass
24,Focus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unclean cclass
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5087,C Class,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unclean focus
5088,C Class,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unclean focus
5089,C Class,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unclean focus
5090,C Class,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unclean focus


In [97]:
df[df['price'].isna()]

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,tax(£),fuel type,engine size,mileage2,fuel type2,engine size2,reference,Class
20,Focus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unclean cclass
21,Focus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unclean cclass
22,Focus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unclean cclass
23,Focus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unclean cclass
24,Focus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unclean cclass
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5087,C Class,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unclean focus
5088,C Class,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unclean focus
5089,C Class,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unclean focus
5090,C Class,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unclean focus


`Looks like it's better to drop off those unclean cclass and unclean focus instances, cause you can see there are no values in any of the columns, and also the author didn't provide few things so yeah let's finish it off...`

In [103]:
df[df['Class'] == 'unclean cclass'].__len__(), df[df['Class'] == 'unclean focus'].__len__()

(4006, 5604)

let's get rid of them instances... because I've thinking what If I delete all the instances according to all the features whose values are nan at that position and let other remain as it is. But some feature don't have single values of this class so yeah let's get rid off these instances though it count for 10% of the data. But not to worry...

In [113]:
# df.drop(labels=['unclean cclass', 'unclean focus'], axis=0, columns=['Class'])

# np.where(df['Class'] == 'unclean class')
# df.loc[:, 'Class']

# The easy way gpt gave me
df = df[df['Class'] != 'unclean cclass']
df = df[df['Class'] != 'unclean focus']


In [121]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 108540 entries, 0 to 15156
Data columns (total 17 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   model         108540 non-null  object 
 1   year          108540 non-null  float64
 2   price         108540 non-null  object 
 3   transmission  108540 non-null  object 
 4   mileage       108540 non-null  object 
 5   fuelType      108540 non-null  object 
 6   tax           94327 non-null   float64
 7   mpg           99187 non-null   float64
 8   engineSize    108540 non-null  float64
 9   tax(£)        4860 non-null    float64
 10  fuel type     0 non-null       object 
 11  engine size   0 non-null       object 
 12  mileage2      0 non-null       object 
 13  fuel type2    0 non-null       object 
 14  engine size2  0 non-null       object 
 15  reference     0 non-null       object 
 16  Class         108540 non-null  object 
dtypes: float64(5), object(12)
memory usage: 14.9+ MB


In [122]:
df['fuel type'].value_counts()

Series([], Name: fuel type, dtype: int64)

Fuck few features came with unclean cclass and unclean focus class... What to do next, I don't know... I'll catch you later😂😂😂😂

11 Features got impacted by deleting those two features... Looks like I've to rollback the decision, so "commenting out above code of removing the classes"